In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('../data/StudentsPerformance.csv')

In [6]:
df.columns = ['gender', 'race', 'parentDegree', 'lunch', 'course', 'mathScore', 'readingScore', 'writingScore']


In [7]:
df['total'] = (df['mathScore']+df['readingScore']+df['writingScore'])/3

In [8]:
for i in range(len(df)):
    if df.iloc[i,2] in ['high school', 'some high school']:
        df.iloc[i,2] = 'No_Degree'
    else:
        df.iloc[i,2] = 'has_Degree'

In [9]:
final_df = df.groupby(['gender','parentDegree','course','lunch','race']).mean().reset_index()
after_sort = final_df.sort_values(by= ['total'],ascending = False)
after_sort.drop(columns=['mathScore','readingScore','writingScore'],inplace = True)
after_sort

,gender,parentDegree,course,lunch,race,total
28,female,has_Degree,completed,standard,group E,84.023810
5,female,No_Degree,completed,standard,group A,82.666667
66,male,has_Degree,completed,standard,group C,79.980392
27,female,has_Degree,completed,standard,group D,79.738095
65,male,has_Degree,completed,standard,group B,79.100000
...,...,...,...,...,...,...
41,male,No_Degree,completed,free/reduced,group C,52.416667
10,female,No_Degree,none,free/reduced,group A,52.066667
12,female,No_Degree,none,free/reduced,group C,50.607843
40,male,No_Degree,completed,free/reduced,group B,50.333333


In [10]:
# Maybe better to move to something else then dummies?

base = pd.get_dummies(final_df,columns=['gender','race','parentDegree','course','lunch'],dtype = int)
base.sample()
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mathScore                79 non-null     float64
 1   readingScore             79 non-null     float64
 2   writingScore             79 non-null     float64
 3   total                    79 non-null     float64
 4   gender_female            79 non-null     int64  
 5   gender_male              79 non-null     int64  
 6   race_group A             79 non-null     int64  
 7   race_group B             79 non-null     int64  
 8   race_group C             79 non-null     int64  
 9   race_group D             79 non-null     int64  
 10  race_group E             79 non-null     int64  
 11  parentDegree_No_Degree   79 non-null     int64  
 12  parentDegree_has_Degree  79 non-null     int64  
 13  course_completed         79 non-null     int64  
 14  course_none              79 

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
train_x,test_x,train_y,test_y = train_test_split(base.iloc[:,4:],base.iloc[:,3],test_size = 0.05)
model = XGBRegressor(max_depth = 6)
model.fit(train_x,train_y)
target = model.predict(test_x)
mean_squared_error(target,test_y)

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/fadykittan/dev/mlops/model_logging/homework/mlops_model_logging_elk/venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <89AD948E-E564-3266-867D-7AF89D6488F0> /Users/fadykittan/dev/mlops/model_logging/homework/mlops_model_logging_elk/venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dyl

In [ ]:
import pickle

In [ ]:
b = pickle.dumps(model)